# CK Flavored S-Expression Interpreter

Scrope to [Playground](#playground) at the bottom to see it in action

**Note**: S-Expressions were chosen because they're really easy to tokenize/parse, and I could skip to grokking interpreters. If you're looking to build your own language, have the parser do more than what is demonstrated below, something akin to what I did in [MinimalMermaidParser.ipynb](../BuildingAParserFromScratch/MinimalMermaidParser.ipynb) would make your life significantly easier, is easier to understand/maintain, etc.

✅ Variables

✅ Simple Functions

✅ Function Closures

✅ 1st Class/Higher Order Functions

✅ Globals (print, concat)

✅ Classes

✅ Class Inheritance

✅ Numbers (Decimal)

✅ Strings

✅ Math Operators (+, -, *, /, %)

⚠️ .NET Interop (Need To Add Generics)

## Prelude

In [1]:
using System.Text.RegularExpressions;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
using System.Reflection;

In [2]:
#r "nuget:Newtonsoft.Json"

Installed Packages Newtonsoft.Json, 13.0.1

In [3]:
#nullable enable

using Newtonsoft.Json;

public class ToStringJsonConverter : JsonConverter
{
    public override bool CanConvert(Type objectType) => true;
    public override bool CanRead => false;

    public override void WriteJson(JsonWriter writer, object? value, JsonSerializer serializer)
    {
        writer.WriteValue(value?.ToString());
    }

    public override object? ReadJson(JsonReader reader, Type objectType, object? existingValue, JsonSerializer serializer)
    {
        return existingValue == null
               ? null
               : Activator.CreateInstance(objectType, new[] { existingValue });
    }
}

public static string ToJson(this object obj)
{
    try
    {
        return JsonConvert.SerializeObject(obj, Formatting.Indented);
    }
    catch (JsonSerializationException)
    {
        return JsonConvert.SerializeObject(obj, Formatting.Indented, new JsonSerializerSettings { PreserveReferencesHandling = PreserveReferencesHandling.Objects });
    }
}

public static DisplayedValue DisplayAsJson(this object obj)
{
    return obj.ToJson().DisplayAs("application/json");
}

## Tokenize S-Expressions

In [4]:
#nullable enable

static IEnumerable<string> Tokenize(string sExpression)
{
    var regex = new Regex(@"[\(|\)]|""[^""]*""|[^\s\(\)]+");
    
    return regex.Matches(sExpression)
                .Cast<Match>()
                .Select(x => x.Value);
}

In [5]:
Tokenize(@"(begin (
    (fun add (x y)
        (+ x y)
    )
))")

index,value
0,(
1,begin
2,(
3,(
4,fun
5,add
6,(
7,x
8,y
9,)


## Parse S-Expression

In [6]:
#nullable enable

// Wrapper for strings so we can differentiate them in our ASTs (Abstract Syntax Trees)
[JsonConverter(typeof(ToStringJsonConverter))]
readonly struct Str
{
    private readonly string _str;
    public Str(string str) => _str = str ?? throw new ArgumentNullException(nameof(str));
    public override string ToString() => _str;
}

// NOTE: This assumes a well formed S-Expression
//       (ex: it does NOT check for balanced parens, etc.)

///<summary>Convert a sequence of tokens to an S-Expression object[] AST</summary>
static object?[] Parse(IEnumerable<string> tokens)
{
    var stack = new Stack<List<object?>>();

    foreach (var token in tokens)
    {
        if (token == "(")
        {
            stack.Push(new List<object?>());
        }
        else if (token == ")")
        {
            var arr = stack.Pop().ToArray();
            
            if (stack.Count == 0)
                return arr;
            
            stack.Peek().Add(arr);
        }
        else
        {
            var value = token == "null"
                        ? null
                        : decimal.TryParse(token, out var number)
                        ? (object)number
                        : token.Length > 1 && token.StartsWith('"') && token.EndsWith('"')
                        ? new Str(token[1..^1])
                        : token;

            stack.Peek().Add(value);
        }
    }
    

    return stack.Peek().ToArray();
}

static object ParseFromString(string str)
{
    var tokens = Tokenize(str);
    var ast = Parse(tokens);
    return ast;
}

In [7]:
ParseFromString("(42)").DisplayAsJson();

ParseFromString("(\"Life, The Universe, And Everything.\")").DisplayAsJson();

ParseFromString(@"(begin
    (fun makeAdder (x)
        (fun capturedAdd (y)
            (+ x y)
        )
    )

    (var myAdder (inv makeAdder (42)))

    (inv myAdder (99)
)").DisplayAsJson();

## Scope

Represents a scope of names and their values (ex: numbers, strings, functions, and classes)

Scopes can be nested (ex: a function has its own scope when invoked, and class instances have their own scopes)

In [8]:
#nullable enable

class Scope
{
    // Using [JsonProperty] so these fields will show up when ToJson() and DisplayAsJson() are called
    [JsonProperty] public readonly Dictionary<string, object?> _data;
    [JsonProperty] private readonly Scope? _parent;

    public Scope(Dictionary<string, object?>? data = null,
                 Scope? parent = null)
    {
        _data = data ?? new Dictionary<string, object?>();
        _parent = parent;
    }

    public void Define(string name, object? value)
    {
        if (_data.ContainsKey(name))
            throw new Exception($"Local scope already contains a name \"{name}\"");

        _data.Add(name, value);
    }

    private Dictionary<string, object?>? FindDictionary(string name, bool throwIfNotFound)
    {
        // Recurse from this scope through
        // ancestor scopes looking for a 
        // Dictionary containing a key name

        var handle = this;
        do
        {
            if (handle._data.ContainsKey(name))
                return handle._data;
        }
        while ((handle = handle._parent) != null);

        if (throwIfNotFound)
            throw new Exception($"No name \"{name}\" found in scope");

        return null;
    }

    public void Set(string name, object? value)
    {
        var data = FindDictionary(name, true)!;
        data[name] = value;
    }

    public object? Get(string name)
    {
        return FindDictionary(name, true)![name];
    }

    public bool Exists(string name)
    {
        return FindDictionary(name, false) is not null;
    }

    public Scope CreateChildScope()
    {
        return new Scope(parent: this);
    }

    public Scope? ParentScope()
    {
        return _parent;
    }
}

## Interpreter

In [9]:
#nullable enable

// Facilitates failing fast which is
// important since this interpreter
// largely traverses object arrays

public static void Expect(object[] ast, string? expectedTag = null, int? minLength = null, int? exactLength = null)
{
    if (ast == null)
        throw new ArgumentNullException(nameof(ast));


    if (expectedTag is not null)
    {
        if (string.IsNullOrWhiteSpace(expectedTag))
            throw new ArgumentOutOfRangeException(nameof(expectedTag), $"If provided, {nameof(expectedTag)} should not be empty or whitespace");
        
        if (expectedTag.Trim() != expectedTag)
            throw new Exception($"{nameof(expectedTag)} should not have leading or trailing whitespace");
        
        if ((ast[0] as string) != expectedTag)
            throw new Exception($"Was expecting tag \"{expectedTag}\" but found \"{ast[0]}\"");
    }


    if (minLength is not null && minLength < 1)
        throw new ArgumentOutOfRangeException(nameof(minLength), $"{minLength} must be null or greater than or equal to one.");

    if (exactLength is not null && exactLength < 1)
        throw new ArgumentOutOfRangeException(nameof(exactLength), $"{exactLength} must be null or greater than or equal to one.");

    if (minLength is not null && exactLength is not null)
        throw new ArgumentOutOfRangeException(nameof(exactLength), $"At most either {nameof(exactLength)} or {nameof(minLength)} should be defined");

    if (minLength is not null && minLength > ast.Length)
        throw new Exception($"Was expecting {nameof(ast)} to be at least of length {minLength} but was of length {ast.Length}");

    if (exactLength is not null && exactLength != ast.Length)
        throw new Exception($"Was expecting {nameof(ast)} to be of length {exactLength} but was of length {ast.Length}");
}

public static T Expect<T>(this object obj)
{
    if (obj is not T)
        throw new Exception($"Was expecting type {typeof(T).Name} but was of type {obj?.GetType().Name ?? "null"}");
    
    return (T)obj;
}

public static T ExpectNotNull<T>(this T? obj)
{
    return obj ?? throw new Exception($"Was expecting non-null {typeof(T).Name} but was null");
}

In [10]:
#nullable enable

class Interpreter
{
    class Function
    {
        public string Name { get; init; }
        public Scope Closure { get; init; }
        public string[] Parameters { get; init; }
        public object Body { get; init; }
    }

    class Member
    {
        public string Name { get; init; }
        public string[] Parameters { get; init; }
        public object[] Body { get; init; }
    }

    class Class
    {
        public string Name { get; init; }
        public string[] FieldNames { get; init; }
        public Member Constructor { get; init; }
        public Dictionary<string, Member> Methods { get; init; }
        public Scope Closure { get; init; }
        public Class? Ancestor { get; init; }
    }

    class ClassInstance
    {
        public Class Class { get; init; }
        public Scope Scope { get; init; }
        public ClassInstance? Parent { get; init; }
    }

    class DotNetClassInstance
    {
        public object Instance { get; init;}
    }

    readonly Scope globalScope;

    static Dictionary<string, Type> dotNetTypes;

    public Interpreter(Scope? globalScope = null)
    {
        this.globalScope = globalScope ?? new Scope();

        // Using an instance to populate a static to be friendly with .NET Interactive
        dotNetTypes = this
                      .GetType()
                      .Assembly
                      .GetReferencedAssemblies()
                      .Select(an => System
                                    .Reflection
                                    .Assembly
                                    .Load(an.FullName))
                      .SelectMany(a => a.GetExportedTypes())
                      .DistinctBy(t => t.FullName)
                      .ToDictionary(t => t.FullName!, t => t);
    }

    static object? Print(object[] ast, Scope scope)
    {
        Expect(ast, "print", minLength: 2);

        object? value = null;
        foreach (var x in ast.Skip(1))
        {
            value = Interpreter.Expression(ast[1], scope);

            if (value == null || value is string || value.GetType().IsPrimitive)
                value.Display();
            else
                value.ToJson().Display();
        }

        return value;
    }

    static object? VariableDeclaration(object[] ast, Scope scope)
    {
        Expect(ast, "var", exactLength: 3);

        var name = ast[1].Expect<string>();
        
        var value = ast.Length == 3
                    ? Expression(ast[2], scope)
                    : null;
        
        scope.Define(name, value);

        return value;
    }

    static object? SetVariable(object[] ast, Scope scope)
    {
        Expect(ast, "set", exactLength: 3);

        var name = ast[1].Expect<string>();
        var value = Expression(ast[2], scope);

        scope.Set(name, value);

        return value;
    }

    static bool Math(string tag, object[] ast, Scope scope, out decimal? result)
    {
        if (!new[] { "+", "-", "*", "/", "%" }.Contains(tag))
        {
            result = null;
            return false;
        }

        if (ast.Length != 3)
            throw new Exception($"{nameof(ast)} for infix math operators is expected to be of length 3 but was of length {ast.Length}");

        var a = Expression(ast[1], scope);
        var b = Expression(ast[2], scope);

        if (!(a is decimal) || !(b is decimal))
            throw new Exception($"Both arguments must be decimals; however, a is {a?.GetType().Name ?? "null"} and b is {b?.GetType().Name ?? "null"}");

        var aDec = a.Expect<decimal>();
        var bDec = b.Expect<decimal>();

        result = tag switch
        {
            "+" => aDec + bDec,
            "-" => aDec - bDec,
            "*" => aDec * bDec,
            "/" => aDec / bDec,
            "%" => aDec % bDec,
            _ => throw new NotImplementedException()
        };

        return true;
    }

    static object? Block(object[] ast, Scope scope)
    {
        Expect(ast, "begin", minLength: 1);
        
        scope = scope.CreateChildScope();

        object? res = null;
        for (int i = 1; i < ast.Length; i++)
            res = Expression(ast[i], scope);

        return res;
    }

    static object? DefineFunction(object[] ast, Scope scope)
    {
        Expect(ast, "fun", minLength: 4);

        var name = ast[1].Expect<string>();
        var parameters = ast[2].Expect<object[]>().Cast<string>().ToArray();
        var body = ast[3];

        if (parameters.Distinct().Count() != parameters.Length)
            throw new Exception("Parameters of Functions must have distinct names within a single Function."); // TODO: Better message

        var function = new Function
        {
            Name = name,
            Closure = scope,
            Parameters = parameters,
            Body = body
        };

        scope.Define(name, function);

        return function;
    }

    static object? InvokeFunction(Function func, object[] args)
    {
        var funcScope = func.Closure.CreateChildScope();

        if (func.Parameters.Length != args.Length)
            throw new Exception($"{func.Name} expects {func.Parameters.Length} argument(s) but was supplied {args.Length}");
        for (int i = 0; i < func.Parameters.Length; i++)
            funcScope.Define(func.Parameters[i], args[i]);

        return Expression(func.Body, funcScope);
    }

    static object? InvokeFunctionOrMethod(object[] ast, Scope scope)
    {
        Expect(ast, "inv", exactLength: 3);

        var name = ast[1].Expect<string>();
        var args = ExpressionArray(ast[2].Expect<object[]>(), scope);

        if (!name.Contains('.'))
        {
            var funcObj = scope.Get(name);
            if (!(funcObj is Function))
                throw new Exception($"Function named \"{name}\" was expected to be in scope; however, \"{name}\" is of type {funcObj?.GetType().Name ?? "null"}");
            var func = (Function)funcObj;

            return InvokeFunction(func, args.Expect<object[]>());
        }
        else
        {
            var nameParts = name.Split('.');
            var instanceName = nameParts[0];
            var methodName = nameParts[1];

            var instance = scope.Get(instanceName);

            switch (instance)
            {
                case ClassInstance classInstance:
                    var method = classInstance.Class.Methods[methodName];
                    return ApplyMember(method, args.Expect<object[]>(), classInstance.Scope);

                case DotNetClassInstance dotNetInstance:
                    var argTypes = args.Select(x => x.GetType()).ToArray();
                    var methodInfo = dotNetInstance.Instance.GetType().GetMethod(methodName, argTypes).ExpectNotNull();
                    return methodInfo.Invoke(dotNetInstance.Instance, args);

                default:
                    throw new NotImplementedException();
            }
        }
    }

    static Class DefineClass(object[] ast, Scope scope)
    {
        Expect(ast, "class", minLength: 5); // 5 for plain classes. 7 for classes that inherit.

        var className = ast[1].Expect<string>();

        var astOffset = 0;
        Class? parent = null;
        if (ast[2] is "extends")
        {
            astOffset = 2;
            parent = scope.Get(ast[3].Expect<string>()).ExpectNotNull().Expect<Class>();
        }

        var fieldsAst = ast[2 + astOffset].Expect<object[]>();
        Expect(fieldsAst, "fields", minLength: 1);
        var fieldNames = fieldsAst.Skip(1).Cast<string>().ToArray();

        var ctorAst = ast[3 + astOffset].Expect<object[]>();
        Expect(ctorAst, "constructor", minLength: 1);
        var ctor = new Member
        {
            Name = "constructor",
            Parameters = ctorAst[1].Expect<object[]>().Cast<string>().ToArray(),
            Body = ctorAst.Skip(2).Cast<object[]>().ToArray()
        };

        var methodsAst = ast[4 + astOffset].Expect<object[]>();
        Expect(methodsAst, "methods", minLength: 1);
        var methods = methodsAst
                      .Skip(1)
                      .Cast<object[]>()
                      .Select(methodAst =>
                      {
                        Expect(methodAst, "mem", minLength: 4);
                        return new Member
                        {
                            Name = methodAst[1].Expect<string>(),
                            Parameters = methodAst[2].Expect<object[]>().Cast<string>().ToArray(),
                            Body = methodAst.Skip(3).ToArray()
                        };
                      })
                      .ToDictionary(m => m.Name,
                                    m => m);

        var @class = new Class
        {
            Name = className,
            FieldNames = fieldNames,
            Constructor = ctor,
            Methods = methods,
            Closure = scope,
            Ancestor = parent
        };

        scope.Define(@class.Name, @class);

        return @class;
    }

    static void PopulateParameters(Member member, object?[] args, Scope memberScope)
    {
        if (member.Parameters.Length != args.Length)
            throw new Exception($"{member.Name} expects {member.Parameters.Length} argument(s) but was supplied {args.Length}");
        
        for (int i = 0; i < member.Parameters.Length; i++)
            memberScope.Define(member.Parameters[i], args[i]);
    }

    static object? ExecuteMemberBody(object[] memberBody, Scope memberScope)
    {
        object? res = null;
        foreach (var exp in memberBody)
            res = Expression(exp, memberScope);
        
        return res;
    }

    static object? ApplyMember(Member member, object[] args, Scope classScope)
    {
        var memberScope = classScope.CreateChildScope();

        PopulateParameters(member, args, memberScope);
        
        return ExecuteMemberBody(member.Body, memberScope);
    }

    static ClassInstance InstantiateClass(Class @class, object?[] ctorArguments, Scope scope)
    {
        ClassInstance? ancestor = null;
        Scope classScope;
        if (@class.Ancestor != null)
        {
            Expect(@class.Constructor.Body, minLength: 1);
            var superAst = @class.Constructor.Body[0].Expect<object[]>();
            Expect(superAst, "super", exactLength: 2);

            var shimScope = @class.Closure.CreateChildScope();
            PopulateParameters(@class.Constructor, ctorArguments, shimScope);

            var parentArgs = ExpressionArray(superAst[1].Expect<object[]>(), shimScope);

            ancestor = InstantiateClass(@class.Ancestor, parentArgs, @class.Ancestor.Closure);
            classScope = ancestor.Scope.CreateChildScope();
        }
        else
        {
            classScope = @class.Closure.CreateChildScope();
        }

        classScope.Define("base", ancestor);
        classScope.Define("this", null);
        foreach (var name in @class.FieldNames)
            classScope.Define(name, null);
        
        var ctorScope = classScope.CreateChildScope();
        PopulateParameters(@class.Constructor, ctorArguments, ctorScope);
        var ctorBody = ancestor != null
                       ? @class.Constructor.Body.Skip(1).ToArray()
                       : @class.Constructor.Body;
        ExecuteMemberBody(ctorBody, ctorScope);

        var instance = new ClassInstance
        {
            Class = @class,
            Scope = classScope
        };
        classScope.Set("this", instance);
        return instance;
    }

    static object InstantiateClass(object[] ast, Scope scope)
    {
        Expect(ast, "new", exactLength: 3);

        var className = ast[1].Expect<string>();

        if (scope.Exists(className))
        {
            var @class = scope.Get(className)!.Expect<Class>();

            var ctorArguments = ExpressionArray(ast[2].Expect<object[]>(), scope);

            return InstantiateClass(@class, ctorArguments, scope);
        }
        else if (dotNetTypes.ContainsKey(className))
        {
            var type = dotNetTypes[className];

            var ctorArguments = ExpressionArray(ast[2].Expect<object[]>(), scope);
            
            var instance = Activator.CreateInstance(type, ctorArguments);

            return new DotNetClassInstance { Instance = instance };
        }
        else
        {
            dotNetTypes.Count.Display();

            throw new Exception($"No class or .NET type found with name \"{className}\"");
        }
    }

    static object? Expression(object? obj, Scope scope)
    {
        if (obj is decimal
            || obj is bool
            || obj is null)
            return obj;

        if (obj is Str)
            return obj.ToString();

        if (obj is string str)
            return scope.Get(str);

        if (obj is object[] arr)
        {
            if (arr.Length == 0)
                return null;

            if (arr[0] is string tag)
            {
                if (Math(tag, arr, scope, out var res))
                    return res;

                if (tag is "print")
                    return Print(arr, scope);

                if (tag is "begin")
                    return Block(arr, scope);

                if (tag is "var")
                    return VariableDeclaration(arr, scope);

                if (tag is "set")
                    return SetVariable(arr, scope);

                if (tag is "get")
                    return scope.Get((string)arr[1]);

                if (tag is "fun")
                    return DefineFunction(arr, scope);

                if (tag is "inv")
                    return InvokeFunctionOrMethod(arr, scope);

                if (tag is "class")
                    return DefineClass(arr, scope);

                if (tag is "new")
                    return InstantiateClass(arr, scope);

                if (tag is "concat")
                    return string.Concat(arr.Skip(1).Select(x => Expression(x, scope)));
            }

            if (arr[0] is object[] innerArr && innerArr.Length == 1)
                return Expression(innerArr, scope);

            if (arr.Length > 1)
                throw new NotImplementedException(arr.ToJson());

            return Expression(arr[0], scope);
        }

        throw new NotImplementedException();
    }

    static object?[] ExpressionArray(object?[] argumentExpressions, Scope scope)
    {
        return argumentExpressions
               .Select(a => Expression(a, scope))
               .ToArray();
    }

    public object? Expression(object obj)
    {
        return Interpreter.Expression(obj, globalScope);
    }

    public static object? Interpret(object ast)
    {
        var interpreter = new Interpreter();
        return interpreter.Expression(ast);
    }

    public static object? Interpret(string str)
    {
        var tokens = Tokenize(str);
        var ast = Parse(tokens);
        return Interpret(ast);
    }
}

## Playground

In [11]:
var sExpressions = new[]
{
    // Numbers
    "(42)",

    // Strings
    "(\"Life, The Universe, And Everything.\")",

    // Blocks
    @"(begin
        (print ""Blocks -------------------------------"")

        (var a ""First"")

        (begin
            (var a ""Second"")
            
            (begin
                (var a ""Third"")    
            )
        )

        a
    )",

    // Simple Functions
    @"(begin
        (print ""Simple Functions ---------------------"")

        (fun aFunction (x y)
            (+ x y)
        )

        (inv aFunction (42 99))
    )",

    // Higher Order Functions
    @"(begin
        (print ""Higher Order Functions ---------------"")

        (fun makeAdder (x)
            (fun capturedAdd (y)
                (+ x y)
            )
        )

        (var myAdder (inv makeAdder (42)))

        (inv myAdder (99))
    )",

    // Classes
    @"(begin
        (print ""Classes ------------------------------"")


        (class Person
            (fields _name _age)
            (constructor (name age)
                (set _name name)
                (set _age age)
            )
            (methods
                (mem GetOlder ()
                    (set _age (+ _age 1))
                )
                (mem ToString ()
                    (concat _name "" is "" _age)
                )
                (mem SomeLiteral ()
                    ""Some Literal""
                )
            )
        )

        (var p (new Person (""Ashley"" 35)))
        (inv p.GetOlder ())
        (print (inv p.ToString ()))
        (inv p.SomeLiteral ())
    )",

    // Inheritance
    @"(begin
        (print ""Inheritance --------------------------"")

        (class Animal
            (fields _name)
            (constructor (name)
                (set _name name)
            )
            (methods
                (mem ToString ()
                    _name
                )
            )
        )

        (var a (new Animal (""Collin"")))
        (print ""// From ToString on Animal"")
        (print (inv a.ToString ()))

        (class Person extends Animal
            (fields _age)
            (constructor (name age)
                (super (name))
                (set _age age)
            )
            (methods
                (mem GetOlder ()
                    (set _age (+ _age 1))
                )
                (mem AnimalToString ()
                    (print ""// From AnimalToString on Person, which calls base.ToString"")
                    (inv base.ToString ())
                )
                (mem ToString ()
                    (print ""// From ToString on Person, which shadows Animal.ToString"")
                    (concat _name "" is "" _age)
                )
            )
        )

        (var p (new Person (""Ashley"" 35)))

        (inv p.GetOlder ())

        (print (inv p.AnimalToString ()))
        
        (inv p.ToString ())
    )",

    // .NET Interop
    @"(begin
        (print "".NET Interop -------------------------"")
    
        (var sb (new System.Text.StringBuilder ()))
        (inv sb.AppendLine (""Hello, World!""))
        (inv sb.Append (""From StringBuilder!""))
    
        (inv sb.ToString ())
    )"
};

foreach (var sExpression in sExpressions)
    Interpreter.Interpret(sExpression).Display();

42

Life, The Universe, And Everything.

Blocks -------------------------------

First

Simple Functions ---------------------

...

Ashley

// From ToString on Person, which shadows Animal.ToString

Ashley is 36

.NET Interop -------------------------

Hello, World!
From StringBuilder!